In [1]:
import pandas as pd
file_1=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Company X - Order Report.xlsx")
file_2=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Company X - Pincode Zones.xlsx")
file_3=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Company X - SKU Master.xlsx")
file_4=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Courier Company - Invoice.xlsx")
file_5=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Courier Company - Rates.xlsx")
file_out_1=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Expected_Result.xlsx",sheet_name='Summary')
file_out_2=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Expected_Result.xlsx",sheet_name='Calculations')

In [2]:
file_4.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [3]:
file_11=file_1.copy()

In [4]:
import numpy as np
import pandas as pd

ID=[i for i in file_1['ExternOrderNo'].unique()]
ID.sort(reverse=False)
ID

file_output=pd.DataFrame(ID,columns={'Order ID'})
print(file_output.shape)
file_output.head()

(124, 1)


,Order ID
0,2001806210
1,2001806226
2,2001806229
3,2001806232
4,2001806233


In [5]:
file_output['Order ID'][18]

2001806652

### Handiling duplicates in data

In [6]:
file_1=file_1.drop_duplicates(ignore_index=True)
file_2=file_2.drop_duplicates(ignore_index=True)
file_3=file_3.drop_duplicates(ignore_index=True)

In [34]:
AWB=[]
TW_X=[]
SLAB_X=[]
TW_C=[]
SLAB_C=[]
ZONE_X=[]
ZONE_C=[]
CHRG_X=[]
CHRG_C=[]
DIFF=[]


for i in range(file_output.shape[0]):
    ID=file_output['Order ID'][i]
    
    # AWB number
    i_4=np.where(file_4['Order ID']==ID)
    i_4=i_4[0][0]
    awb=file_4['AWB Code'][i_4]
    AWB.append(awb)
    
    # total weight of order as per x
    i_1=np.where(file_1['ExternOrderNo']==ID)
    i_1=i_1[0]  #bunch of index of similar oderID
    skw=0
    for j in i_1:
        sk=file_1['SKU'][j]
        qt=file_1['Order Qty'][j]
        i_3=np.where(file_3['SKU']==sk)
        i_3=i_3[0][0]
        sk_w=file_3['Weight (g)'][i_3]
        sk_w=sk_w/1000
        sk_w=sk_w*qt #### sku (product) induviudal weight mutiplied by quentity sku
        sk_w=skw+sk_w
        skw=sk_w
    TW_X.append(skw)
    
    # weight slab as per x
    inte=skw//1
    fra=skw%1
    if fra==0:
        SLAB_X.append(inte)
    elif fra>0 and fra<=0.5:
        SLAB_X.append(inte+0.5)
    else:
        SLAB_X.append(inte+1)
    
    # total weight of order as per c
    tw_c=file_4['Charged Weight'][i_4]
    TW_C.append(tw_c)
    
    # weight slab as per x
    inte=tw_c//1
    fra=tw_c%1
    if fra==0:
        sl_x=inte
        SLAB_C.append(sl_x)
    elif fra>0 and fra<=0.5:
        sl_x=inte+0.5
        SLAB_C.append(sl_x)
    else:
        sl_x=inte+1
        SLAB_C.append(sl_x)
        
    # Delivery zone as per x
    pin_x=file_4['Customer Pincode'][i_4]
    i_2=np.where(file_2['Customer Pincode']==pin_x)
    i_2=i_2[0][0]
    z_x=file_2['Zone'][i_2]
    z_x=z_x.upper()
    ZONE_X.append(z_x)
    
    #Delivery zone as per c
    z_c=file_4['Zone'][i_4]
    z_c=z_c.upper()
    ZONE_C.append(z_c)
    
    #Expected Charge as per X (Rs.)
    ts=file_4['Type of Shipment'][i_4] ##Type of Shipment    
    if ts=='Forward charges':
        if z_x=='A':
            if sl_x<=0.5:
                chrg=file_5['fwd_a_fixed']
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=file_5['fwd_a_fixed']+((no_sl-1)*(file_5['fwd_a_additional']))
                CHRG_X.append(chrg)
        elif z_x=='B':
            if sl_x<=0.5:
                chrg=file_5['fwd_b_fixed'] ###########################
                print(chrg)
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=file_5['fwd_b_fixed']+((no_sl-1)*(file_5['fwd_b_additional']))
                CHRG_X.append(chrg)
        elif z_x=='C':
            if sl_x<=0.5:
                chrg=file_5['fwd_c_fixed']
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=file_5['fwd_c_fixed']+((no_sl-1)*(file_5['fwd_c_additional']))
                CHRG_X.append(chrg)
        elif z_x=='D':
            if sl_x<=0.5:
                chrg=file_5['fwd_d_fixed']
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=file_5['fwd_d_fixed']+((no_sl-1)*(file_5['fwd_d_additional']))
                CHRG_X.append(chrg)
        else:
            if sl_x<=0.5:
                chrg=file_5['fwd_e_fixed']
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=file_5['fwd_e_fixed']+((no_sl-1)*(file_5['fwd_e_additional']))
                CHRG_X.append(chrg)
                ######################################################################
    else:
        if z_x=='A':
            if sl_x<=0.5:
                chrg=file_5['fwd_a_fixed']+file_5['rto_a_fixed']
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=(file_5['fwd_a_fixed']+file_5['rto_a_fixed'])+((no_sl-1)*(file_5['fwd_a_additional']+file_5['rto_a_additional']))
                CHRG_X.append(chrg)
        elif z_x=='B':
            if sl_x<=0.5:
                chrg=file_5['fwd_b_fixed']+file_5['rto_b_fixed']
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=(file_5['fwd_b_fixed']+file_5['rto_b_fixed'])+((no_sl-1)*(file_5['fwd_b_additional']+file_5['rto_b_additional']))
                CHRG_X.append(chrg)
        elif z_x=='C':
            if sl_x<=0.5:
                chrg=file_5['fwd_c_fixed']+file_5['rto_c_fixed']
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=(file_5['fwd_c_fixed']+file_5['rto_c_fixed'])+((no_sl-1)*(file_5['fwd_c_additional']+file_5['rto_c_additional']))
                CHRG_X.append(chrg)
        elif z_x=='D':
            if sl_x<=0.5:
                chrg=file_5['fwd_d_fixed']+file_5['rto_d_fixed']
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=(file_5['fwd_d_fixed']+file_5['rto_d_fixed'])+((no_sl-1)*(file_5['fwd_d_additional']+file_5['rto_d_additional']))
                CHRG_X.append(chrg)
        else:
            if sl_x<=0.5:
                chrg=file_5['fwd_e_fixed']+file_5['rto_e_fixed']
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=(file_5['fwd_e_fixed']+file_5['rto_e_fixed'])+((no_sl-1)*(file_5['fwd_e_additional']+file_5['rto_e_additional']))
                CHRG_X.append(chrg)
             
    # Charges Billed by Courier Company (Rs.)
    chrg_c=file_4['Billing Amount (Rs.)'][i_4]
    CHRG_C.append(chrg_c)
    
    # Difference Between Expected Charges and Billed Charges (Rs.)
    diff=chrg-chrg_c
    DIFF.append(diff)
    
    
### Requred Modificartion ###
CHRG_X_NEW=[]
DIFF_NEW=[]
for i in range(len(CHRG_X)):
    chrg_new=CHRG_X[i][0]
    CHRG_X_NEW.append(chrg_new)
    
for i in range(len(CHRG_X)):
    diff_new=CHRG_X_NEW[i]-CHRG_C[i]
    DIFF_NEW.append(diff_new)

0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64
0    33
Name: fwd_b_fixed, dtype: int64


In [32]:
out=[AWB,AWB,TW_X,SLAB_X,TW_C,SLAB_C,ZONE_X,ZONE_C,CHRG_X_NEW,CHRG_C,DIFF_NEW]
for i,j in zip(file_out_2.columns,out):
    if i=='Order ID':
        pass
    else:
        file_output[i]=j


In [33]:
file_output

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,B,B,0 174.5 dtype: float64,174.5,0 0.0 dtype: float64
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,D,D,0 90.2 dtype: float64,90.2,0 -1.421085e-14 dtype: float64
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,D,D,0 90.2 dtype: float64,90.2,0 -1.421085e-14 dtype: float64
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,0 135.0 dtype: float64,135.0,0 0.0 dtype: float64
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,B,B,0 61.3 dtype: float64,61.3,0 0.0 dtype: float64
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,0.5,B,D,33,45.4,-12.4
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,B,D,0 89.6 dtype: float64,135.0,0 -45.4 dtype: float64
121,2001823564,1091121666133,0.672,1.0,0.70,1.0,D,D,0 176.3 dtype: float64,172.8,0 3.5 dtype: float64
122,2001825261,1091121981575,1.557,2.0,1.60,2.0,D,D,0 355.5 dtype: float64,345.0,0 10.5 dtype: float64


In [28]:
file_5['fwd_b_fixed'][0]

33

In [25]:
CHRG_X

[0    174.5
 dtype: float64,
 0    90.2
 dtype: float64,
 0    90.2
 dtype: float64,
 0    135.0
 dtype: float64,
 0    61.3
 dtype: float64,
 0    135.0
 dtype: float64,
 0    90.2
 dtype: float64,
 0    61.3
 dtype: float64,
 0    90.2
 dtype: float64,
 0    224.6
 dtype: float64,
 0    90.2
 dtype: float64,
 0    61.3
 dtype: float64,
 0    117.9
 dtype: float64,
 0    90.2
 dtype: float64,
 0    61.3
 dtype: float64,
 0    135.0
 dtype: float64,
 0    61.3
 dtype: float64,
 0    89.6
 dtype: float64,
 0    90.2
 dtype: float64,
 0    45.4
 Name: fwd_d_fixed, dtype: float64,
 0    90.2
 dtype: float64,
 0    135.0
 dtype: float64,
 0    90.2
 dtype: float64,
 0    89.6
 dtype: float64,
 0    174.5
 dtype: float64,
 0    89.6
 dtype: float64,
 0    61.3
 dtype: float64,
 0    61.3
 dtype: float64,
 0    61.3
 dtype: float64,
 0    61.3
 dtype: float64,
 0    90.2
 dtype: float64,
 0    45.4
 Name: fwd_d_fixed, dtype: float64,
 0    179.8
 dtype: float64,
 0    89.6
 dtype: float64,
 

In [ ]:
file_11.rename(columns = {'w':'Tot.w'}, inplace = True)
file_11['Tot.w']=file_11['Tot.w']*file_11['Order Qty']
file_11

In [ ]:
gr=file_11.groupby('ExternOrderNo',as_index=False)

In [ ]:
file_output=gr.agg(sum)
file_output.rename(columns={'ExternOrderNo':'Order ID'},inplace=True)
AWB=[]
for i in range(file_output.shape[0]):
    ID=file_output['Order ID'][i]
    i_1=np.where(file_4['Order ID']==ID)
    i_1=i_1[0][0]
    aw=file_4['AWB Code'][i_1]
    AWB.append(aw)
file_output.insert(1,'AWB Code',AWB)

file_output.rename(columns={'Tot.w':'Total weight as per X (KG)'},inplace=True)
file_output.drop('Order Qty',axis=1,inplace=True)
file_output

In [ ]:
slab=[]
for i in file_output['Total weight as per X (KG)']:
    inte=i//1
    fra=i%1
    if fra==0:
        slab.append(inte)
    elif fra>0 and fra<=0.5:
        slab.append(inte+0.5)
    else:
        slab.append(inte+1)
file_output['Weight slab as per X (KG)']=slab
file_output

In [ ]:
TWC=[]
for i in range(file_output.shape[0]):
    ID=file_output['Order ID'][i]
    i_1=np.where(file_4['Order ID']==ID)
    i_1=i_1[0][0]
    twc=file_4['Charged Weight'][i_1]
    TWC.append(twc)
file_output['Total weight as per Courier Company (KG)']=TWC
file_output

In [ ]:
slab_C=[]
for i in file_output['Total weight as per Courier Company (KG)']:
    inte=i//1
    fra=i%1
    if fra==0:
        slab_C.append(inte)
    elif fra>0 and fra<=0.5:
        slab_C.append(inte+0.5)
    else:
        slab_C.append(inte+1)
file_output['Weight slab charged by Courier Company (KG)']=slab_C
file_output.head()

In [ ]:
ZONE_X=[]
ZONE_C=[]
for i in range(file_output.shape[0]):
    ID=file_output['Order ID'][i]
    i_1=np.where(file_4['Order ID']==ID)
    i_1=i_1[0][0]
    c_pin=file_4['Customer Pincode'][i_1]
    zone_c=file_4['Zone'][i_1]
    ZONE_C.append(zone_c)
    i_2=np.where(file_2['Customer Pincode']==c_pin)
    i_2=i_2[0][0]
    zone_x=file_2['Zone'][i_2]
    ZONE_X.append(zone_x)
file_output['Delivery Zone as per X']=ZONE_X
file_output['Delivery Zone charged by Courier Company']=ZONE_C
file_output

In [ ]:
CHRG_X=[]
for i in range(file_output.shape[0]):
    ID=file_output['Order ID'][i]
    i_1=np.where(file_4['Order ID']==ID)
    i_1=i_1[0][0]
    s_type=file_4['Type of Shipment'][i_1]
    if s_type=='Forward charges':
        if file_output['Delivery Zone as per X'][i]=='a':
            if file_output['Weight slab as per X (KG)'][i]<=0.5:
                chrg_x=file_5['fwd_a_fixed'][0]
                CHRG_X.append(chrg_x)
            else:
                sl=file_output['Weight slab as per X (KG)'][i]*2
                chrg_x=(file_5['fwd_a_fixed'][0])+((sl-1)*(file_5['fwd_a_additional'][0]))
                CHRG_X.append(chrg_x)
        elif file_output['Delivery Zone as per X'][i]=='b':
            if file_output['Weight slab as per X (KG)'][i]<=0.5:
                chrg_x=file_5['fwd_b_fixed'][0]
                CHRG_X.append(chrg_x)
            else:
                sl=file_output['Weight slab as per X (KG)'][i]*2
                chrg_x=(file_5['fwd_b_fixed'][0])+((sl-1)*(file_5['fwd_b_additional'][0]))
                CHRG_X.append(chrg_x)
        elif file_output['Delivery Zone as per X'][i]=='c':
            if file_output['Weight slab as per X (KG)'][i]<=0.5:
                chrg_x=file_5['fwd_c_fixed'][0]
                CHRG_X.append(chrg_x)
            else:
                sl=file_output['Weight slab as per X (KG)'][i]*2
                chrg_x=(file_5['fwd_c_fixed'][0])+((sl-1)*(file_5['fwd_c_additional'][0]))
                CHRG_X.append(chrg_x)
        elif file_output['Delivery Zone as per X'][i]=='d':
            if file_output['Weight slab as per X (KG)'][i]<=0.5:
                chrg_x=file_5['fwd_d_fixed'][0]
                CHRG_X.append(chrg_x)
            else:
                sl=file_output['Weight slab as per X (KG)'][i]*2
                chrg_x=(file_5['fwd_d_fixed'][0])+((sl-1)*(file_5['fwd_d_additional'][0]))
                CHRG_X.append(chrg_x)
        else:
            if file_output['Weight slab as per X (KG)'][i]<=0.5:
                chrg_x=file_5['fwd_e_fixed'][0]
                CHRG_X.append(chrg_x)
            else:
                sl=file_output['Weight slab as per X (KG)'][i]*2
                chrg_x=(file_5['fwd_e_fixed'][0])+((sl-1)*(file_5['fwd_e_additional'][0]))
                CHRG_X.append(chrg_x)
    else:
        if file_output['Delivery Zone as per X'][i]=='a':
            if file_output['Weight slab as per X (KG)'][i]<=0.5:
                chrg_x=(file_5['fwd_a_fixed'][0])+(file_5['rto_a_fixed'][0])
                CHRG_X.append(chrg_x)
            else:
                sl=file_output['Weight slab as per X (KG)'][i]*2
                chrg_x=((file_5['fwd_a_fixed'][0])+(file_5['rto_a_fixed'][0]))+((sl-1)*((file_5['fwd_a_additional'][0])+(file_5['rto_a_additional'][0])))
                CHRG_X.append(chrg_x)
        elif file_output['Delivery Zone as per X'][i]=='b':
            if file_output['Weight slab as per X (KG)'][i]<=0.5:
                chrg_x=(file_5['fwd_b_fixed'][0])+(file_5['rto_b_fixed'][0])
                CHRG_X.append(chrg_x)
            else:
                sl=file_output['Weight slab as per X (KG)'][i]*2
                chrg_x=((file_5['fwd_b_fixed'][0])+(file_5['rto_b_fixed'][0]))+((sl-1)*((file_5['fwd_b_additional'][0])+(file_5['rto_b_additional'][0])))
                CHRG_X.append(chrg_x)
        elif file_output['Delivery Zone as per X'][i]=='c':
            if file_output['Weight slab as per X (KG)'][i]<=0.5:
                chrg_x=(file_5['fwd_c_fixed'][0])+(file_5['rto_c_fixed'][0])
                CHRG_X.append(chrg_x)
            else:
                sl=file_output['Weight slab as per X (KG)'][i]*2
                chrg_x=((file_5['fwd_c_fixed'][0])+(file_5['rto_c_fixed'][0]))+((sl-1)*((file_5['fwd_c_additional'][0])+(file_5['rto_c_additional'][0])))
                CHRG_X.append(chrg_x)
        elif file_output['Delivery Zone as per X'][i]=='d':
            if file_output['Weight slab as per X (KG)'][i]<=0.5:
                chrg_x=(file_5['fwd_d_fixed'][0])+(file_5['rto_d_fixed'][0])
                CHRG_X.append(chrg_x)
            else:
                sl=file_output['Weight slab as per X (KG)'][i]*2
                chrg_x=((file_5['fwd_d_fixed'][0])+(file_5['rto_d_fixed'][0]))+((sl-1)*((file_5['fwd_d_additional'][0])+(file_5['rto_d_additional'][0])))
                CHRG_X.append(chrg_x)
        else:
            if file_output['Weight slab as per X (KG)'][i]<=0.5:
                chrg_x=(file_5['fwd_e_fixed'][0])+(file_5['rto_e_fixed'][0])
                CHRG_X.append(chrg_x)
            else:
                sl=file_output['Weight slab as per X (KG)'][i]*2
                chrg_x=((file_5['fwd_e_fixed'][0])+(file_5['rto_e_fixed'][0]))+((sl-1)*((file_5['fwd_e_additional'][0])+(file_5['rto_e_additional'][0])))
                CHRG_X.append(chrg_x)
file_output['Expected Charge as per X (Rs.)']=CHRG_X
file_output

In [ ]:
CHRG_C=[]
for i in range(file_output.shape[0]):
    ID=file_output['Order ID'][i]
    i_1=np.where(file_4['Order ID']==ID)
    i_1=i_1[0][0]
    s_type=file_4['Type of Shipment'][i_1]
    if s_type=='Forward charges':
        if file_output['Delivery Zone charged by Courier Company'][i]=='a':
            if file_output['Weight slab charged by Courier Company (KG)'][i]<=0.5:
                chrg_c=file_5['fwd_a_fixed'][0]
                CHRG_C.append(chrg_x)
            else:
                sl=file_output['Weight slab charged by Courier Company (KG)'][i]*2
                chrg_c=(file_5['fwd_a_fixed'][0])+((sl-1)*(file_5['fwd_a_additional'][0]))
                CHRG_C.append(chrg_c)
        elif file_output['Delivery Zone charged by Courier Company'][i]=='b':
            if file_output['Weight slab charged by Courier Company (KG)'][i]<=0.5:
                chrg_c=file_5['fwd_b_fixed'][0]
                CHRG_C.append(chrg_c)
            else:
                sl=file_output['Weight slab charged by Courier Company (KG)'][i]*2
                chrg_c=(file_5['fwd_b_fixed'][0])+((sl-1)*(file_5['fwd_b_additional'][0]))
                CHRG_C.append(chrg_c)
        elif file_output['Delivery Zone charged by Courier Company'][i]=='c':
            if file_output['Weight slab charged by Courier Company (KG)'][i]<=0.5:
                chrg_c=file_5['fwd_c_fixed'][0]
                CHRG_C.append(chrg_c)
            else:
                sl=file_output['Weight slab charged by Courier Company (KG)'][i]*2
                chrg_c=(file_5['fwd_c_fixed'][0])+((sl-1)*(file_5['fwd_c_additional'][0]))
                CHRG_C.append(chrg_x)
        elif file_output['Delivery Zone charged by Courier Company'][i]=='d':
            if file_output['Weight slab charged by Courier Company (KG)'][i]<=0.5:
                chrg_x=file_5['fwd_d_fixed'][0]
                CHRG_C.append(chrg_c)
            else:
                sl=file_output['Weight slab charged by Courier Company (KG)'][i]*2
                chrg_c=(file_5['fwd_d_fixed'][0])+((sl-1)*(file_5['fwd_d_additional'][0]))
                CHRG_C.append(chrg_c)
        else:
            if file_output['Weight slab charged by Courier Company (KG)'][i]<=0.5:
                chrg_c=file_5['fwd_e_fixed'][0]
                CHRG_C.append(chrg_c)
            else:
                sl=file_output['Weight slab charged by Courier Company (KG)'][i]*2
                chrg_c=(file_5['fwd_e_fixed'][0])+((sl-1)*(file_5['fwd_e_additional'][0]))
                CHRG_C.append(chrg_c)
    else:
        if file_output['Delivery Zone charged by Courier Company'][i]=='a':
            if file_output['Weight slab charged by Courier Company (KG)'][i]<=0.5:
                chrg_c=(file_5['fwd_a_fixed'][0])+(file_5['rto_a_fixed'][0])
                CHRG_C.append(chrg_c)
            else:
                sl=file_output['Weight slab charged by Courier Company (KG)'][i]*2
                chrg_c=((file_5['fwd_a_fixed'][0])+(file_5['rto_a_fixed'][0]))+((sl-1)*((file_5['fwd_a_additional'][0])+(file_5['rto_a_additional'][0])))
                CHRG_C.append(chrg_c)
        elif file_output['Delivery Zone charged by Courier Company'][i]=='b':
            if file_output['Weight slab charged by Courier Company (KG)'][i]<=0.5:
                chrg_c=(file_5['fwd_b_fixed'][0])+(file_5['rto_b_fixed'][0])
                CHRG_C.append(chrg_c)
            else:
                sl=file_output['Weight slab charged by Courier Company (KG)'][i]*2
                chrg_c=((file_5['fwd_b_fixed'][0])+(file_5['rto_b_fixed'][0]))+((sl-1)*((file_5['fwd_b_additional'][0])+(file_5['rto_b_additional'][0])))
                CHRG_C.append(chrg_c)
        elif file_output['Delivery Zone charged by Courier Company'][i]=='c':
            if file_output['Weight slab charged by Courier Company (KG)'][i]<=0.5:
                chrg_c=(file_5['fwd_c_fixed'][0])+(file_5['rto_c_fixed'][0])
                CHRG_C.append(chrg_c)
            else:
                sl=file_output['Weight slab charged by Courier Company (KG)'][i]*2
                chrg_c=((file_5['fwd_c_fixed'][0])+(file_5['rto_c_fixed'][0]))+((sl-1)*((file_5['fwd_c_additional'][0])+(file_5['rto_c_additional'][0])))
                CHRG_C.append(chrg_c)
        elif file_output['Delivery Zone charged by Courier Company'][i]=='d':
            if file_output['Weight slab charged by Courier Company (KG)'][i]<=0.5:
                chrg_c=(file_5['fwd_d_fixed'][0])+(file_5['rto_d_fixed'][0])
                CHRG_C.append(chrg_c)
            else:
                sl=file_output['Weight slab charged by Courier Company (KG)'][i]*2
                chrg_c=((file_5['fwd_d_fixed'][0])+(file_5['rto_d_fixed'][0]))+((sl-1)*((file_5['fwd_d_additional'][0])+(file_5['rto_d_additional'][0])))
                CHRG_C.append(chrg_c)
        else:
            if file_output['Weight slab charged by Courier Company (KG)'][i]<=0.5:
                chrg_c=(file_5['fwd_e_fixed'][0])+(file_5['rto_e_fixed'][0])
                CHRG_C.append(chrg_c)
            else:
                sl=file_output['Weight slab charged by Courier Company (KG)'][i]*2
                chrg_c=((file_5['fwd_e_fixed'][0])+(file_5['rto_e_fixed'][0]))+((sl-1)*((file_5['fwd_e_additional'][0])+(file_5['rto_e_additional'][0])))
                CHRG_C.append(chrg_c)
file_output['Charges Billed by Courier Company (Rs.)']=CHRG_C
file_output

In [ ]:
file_output['Difference Between Expected Charges and Billed Charges (Rs.)']=file_output['Expected Charge as per X (Rs.)']-file_output['Charges Billed by Courier Company (Rs.)']

In [ ]:
file_output

In [ ]:
file_output.to_excel(r'C:\Users\Parimal\Desktop\final.xlsx')

In [ ]:
a=file_4['Zone'][0]
a=a.upper()

In [ ]:
a